<span type="title">初始化和清理</span> | <span type="update">2018-07-17</span> - Version <span type="version">1.6</span>
    
    
<span type="intro"><p class="card-text">本章主要介绍Java的对象初始化和清理相关内容。在初始化部分，介绍了使用构造器进行方法重载和对象初始化、使用this关键字进行协同构造和作为方法返回值的技巧，讲解了变量初始化和数组初始化以及其初始化顺序问题，同时讲解了可变参数列表、enum枚举类型等知识点。在清理部分，讲解了finalize方法和Java内存清理的逻辑问题，比如 stop-copy 和 mark-sweep 策略。</p></span>

# 对象初始化

Java的对象会进行初始化，当你调用 `String s = new String();` 的时候，通过`new`已经调用了String类的构造器进行初始化，默认的，当你自己构造类的时候，可以不提供初始化方法，Java会自动进行，但是，也不能传递参数。比如：

```java
class Rock {};
Rock r0 = Rock();//不能写任何参数
```

## 构造器初始化

更加普遍的，初始化对象使用构造器进行，其采用一个和类一样名字的方法作为语法。

```java
class Rock {
	Rock(){
		System.out.println("Initlizing Rock Object...");
	}
	Rock(int id){
		System.out.println("Initlizing Rock with id "+id);
	}
	Rock(int id,String name){
		System.out.println("Initlizing Rock with id "+id+" ,name is "+ name);
	}
	Rock(String name,int id){
		System.out.println("Initlizing Rock with id "+id+" ,name is "+ name);
	}
}
Rock r = new Rock();
Rock r2 = new Rock(233);
Rock r3 = new Rock(233,"Corkine");
Rock r4 = new Rock("Corkine",233);
```

构造器没有返回值，可以有参数，参数在调用new进行对象构造的时候进行设置。


## 方法重载

构造器可以进行方法重载，区分不同构造器的方法是：看传入参数的不同，比如你可以传入一个int，String，传入参数类型、个数、顺序都用来区别不同的构造器。需要注意的是，方法重载可以重载任意的方法，比如：

```java
class Dog {
	void bark(){
		System.out.println("Bark with nothing...");
	}
	void bark(int n){
		System.out.println("Bark with number "+n);
	}
	void bark(String s){
		System.out.println("Bark with string "+s);
		return;
	}
}
Dog a = new Dog(), b = new Dog();
a.bark(1);
b.bark("a");
b.brak(1);
```

## this 指代

对象在其内部有一个用来指代自身的关键字 `this`。一般而言，其用来**作为方法的返回值**，这样就可以多次使用点运算符调用方法。如下`Leaf`的`increment()`方法所示。

```java
class Leaf {
	int i = 0;
	Leaf increment(){
		i++;
		return this;
	}
	void result(){
		System.out.println("The Leaf now is "+i);
	}
}
Leaf e1 = new Leaf();
e1.increment().increment().increment().result();//The Leaf now is 3
```

此外，this还可以用作在其内部调用变量和方法的前缀，虽然毫无用处（直接调用变量或者方法即可，不用加this.）。

注意下面这个例子，this的第二个用途在于**重载构造器**，可以在一个构造器内部重载第二个构造器，但是需要注意，构造器重载在外部构造器里只能进行一次，并且只能放在首行。

```java
class Flower{
	String info = "NaN";
	Flower(int a){
		System.out.println("Now init Flower with int "+a);
	}
	Flower(String s, int a){
		System.out.println("Now init Flower with string "+s+", int "+a);
	}
	Flower(int a, String s){
		//System.out.println("Wait, I will call the host now"); //can't be here!!!
        this(s,a);//can't call at line 2, can't call 2, can't call self
		this.info = "Nope";
		System.out.println("Wait, I called the host for you!");
	}
}
Flower f = new Flower(27,"rose");
Flower f2 = new Flower("rose",28);
System.out.println(f.info+" "+f2.info);
```

**this 和 static**

this的使用可以让我们清楚的看到static意味着什么，static意味着没有this的方法或者变量，static是非面向对象的写法，所有由同一个类创建的static享有同一片内存地址，越少创建static越好。

## 变量初始化

Java会保证类的成员在初始化的时候被赋值，但是不保证方法的成员初始化。

```java
class A {
    int a;
    float b;
    void f(){
        int c;
        //c++; error
    }
}
A a = new A();
System.out.print(a.a);//0
System.out.print(a.b);//0.0
a.f() //error
```

```java
class A{
	static int q = 0;
	static int i;
	static {
		i = 1;
		//boolean b = false;//can't find A.b but A.i
		//must define out of static context
	}
	int z;
	{
		i = 30;//if A() IS nothing, the final ans will be 30
		//int z = 999;// error
		z = 999;
	}
	A(){
		i = 27; //after static load, so this is the final answer
		//int i = 27; can't work
	}
	
}
A a = new A();
System.out.println(""+A.i+" ,"+a.z); //27， 999

```

**静态变量初始化**

如上代码，采用 `static {}` 的方式，可以声明一个 静态变量上下文，在此进行静态变量的初始化。当然，你也可以直接 `static int q = 0` 这样对每个变量初始化。需要注意，不能在 静态变量上下文 中定义变量，只能用来赋值。

**动态变量初始化**

对于动态变量而言，其也可以单独今昔能够，比如 `int z = 0;` 也可以使用花括号括起来，需要注意，在花括号内部只能赋值，而不能定义变量。

**初始化顺序问题**

很显然的，静态变量最优先被初始化，接着是动态变量，不论其在代码的任何位置。之后是构造器，构造器之后才是方法。因此，在上述的代码中，结果i为27。

## 域初始化

当变量初始化时，我们看到那些写在 `static { }` 域内的东西。一般而言，这里放置的是定义但是未赋值的一组数据的值，对这些数据在同一个位置进行赋值。但是，域 `{}` 放置在类中，不仅仅能够做到这些。静态域可以执行语句，在类被加载并且初始化的时候，在构造器构造之前。动态域也是。有些时候，我们需要执行一个静态方法，那么将这个方法放置到静态域中，就可以进行初始化。比如：

```java
private static Map m;
private static void loader() {
    m = new HashMap<>();
    for (Class<? extends Pet> t : LiteralPetCreator.allTypes){
        m.put(t,0);
    }
}
static { loader();} 
//如果想要将一个方法在初始化时加载，这样做，这个方法处理的数据必须保存在一个static中
```

可以看到，在这个类被载入内存的时候，进行了 loader() 方法的调用。而这个方法，则更改了静态变量 m 的值。因为 m 代码在前，因此先执行，所以这里会直接给 m 赋值。这就是由于静态域的高优先级，所以将那些需要在初始化类时，构造器之前就确定的东西放置到静态域中执行。

此外，动态域 `{ }` 的使用更方便，这里面不需要方法是静态的，你可以写任何语句，在初始化的时候（类的静态方法、字段被调用的时候或者构造器被调用的时候），这些语句会执行。这些语句是在编译期之后执行的，相关内容参见RTTI。

此外，静态域初始化的一个作用是解决这样的问题：

```java
public PetCounter() {
    super(m) }
}
```

因为super()的调用必须放在第一句，但是我们需要在这个时候就初始化好了这个 m 对象，因此使用域包裹代码执行，因为构造器调用顺序在域之后，因此就可以顺利的将参数传递给 super 了。

## 数组初始化

```java
public static void main(String[] args){
    int[] list_i = {1,2,3,4,5};//base type
    int[] list_p = new int[]{1,2,3,4,5};//base type
    Integer[] list_q = new Integer[]{1,2,3,4,5};//cons type
    String[] list_a = new String[]{"hello","world","from","cm"};
    for (String s: list_a){
        System.out.println(s);
    }
    for (int i:list_i){
        System.out.println(i);
    }
    for (int q:list_q){
        System.out.println(q);
    }
}

```

上述代码介绍了数组初始化的过程，首先，其定义为 `int[]` 表示数组类型，接着，创建包装器对象或者直接赋值，前者保存在堆中，后者保存在堆栈中。可以在初始化过程中就进行赋值。

需要注意，使用包装器 `Integer[]` 和使用基本类型 `int[]` 创建的数组不同，前者是引用数组，后者是包含值的数组。需要注意，使用 `int[] a = new int[]{1,2,3}` 创建的数组默认会被转换成 `int[] a = {1,2,3}` 这样的实数数组。

## 可变参数列表

```java
static void printArray(Object[] args){
    for (Object obj : args){
        System.out.print(obj+" ");
    }
}
static void f(String...args){
    for (String s : args){
        System.out.print(s+" ");
    }
}
static void q(Integer...args){
    for (Integer s : args){
        System.out.print(s+" ");
    }
}
static void p(int...args){
    for (int s : args){
        System.out.print(s+" ");
    }
}

printArray(new Object[]{1,2,3,4,new String("hello"),"cm"});
f("hello","cm");
q(1,2,3);
q(new Integer(1),new Integer(2),new Integer(3));
p(1,2,3);
p(new Integer(1),new Integer(2),new Integer(3));
```

如上所述，列表可以用作参数，比如 `printArray` 所示，这是一个Object类型的数组，这意味着其可以接受任何类型构造器创建的引用。

此外，可以使用可变参数列表，虽然这可能导致了方法重载时的混淆，但是毕竟很方便，需要注意，这样的列表的类型限定为完全一样。使用 `type...name` 来表示这个数组作为参数的语法，调用名称即可调用数组。

需要注意，在上述代码中，需要Integer的q方法在接受int类型参数的时候，将其自动转换成了Integer类型，类似于`int a = 4;Integer a = a;`。

## 枚举关键字

```java
public static void main(String[] args){
    Person tom = Person.ONE;
    System.out.println(tom);//toString() Method
    for (Person p : Person.values()){
        System.out.println(p + " " + p.ordinal());
    }
    switch (tom) {
        case ONE:
        case FOUR:
            System.out.println("Got it");
            break;
        default:
            System.out.println("Nope!");
    }
}
public enum Person {
    ONE, TWO, THREE, FOUR
}
```

enum 作为枚举关键字，非常方便。如上的代码所示，枚举类型自带了toString()方法，并且含有values()方法来进行遍历，还有ordinal()方法来指明其顺序。更好玩的是，其可以使用switch语句进行多重判断。

# 对象的回收和清理

Java 的回收是自动进行的。默认里，不用使用 finalized() 进行任何工作，当指向对象的引用超出作用域，其就被垃圾回收掉。但是，重载 finalized 在某些情况下可能有用，比如Java嵌套了其余语言的代码，生成一堆不能被垃圾回收器回收的局部变量，比如，需要在对象回收前进行某些工作，比如关闭文件流等。如下：

```java
class Person{
	int id = 0;
	boolean isvip = false;
	protected void finalize() throws Throwable{
		System.out.println("Closing door! Checking VIP");
		if (isvip) {
			System.out.println("I'm VIP!!! You can't stop me!");	
		} else {
			System.out.println("You're not a vip, Bye!");	
			super.finalize();
		}
	}
}
Person tom = new Person(), marvin = new Person();
tom.isvip = true;
System.gc();//强制垃圾回收
```

这段代码重载了子类的 finalize() 方法，并且强制垃圾回收，当进行回收的时候，如果是 vip，那么就不回收，否则调用父类的 finalize 进行回收。



Java 的回收有两种方式：stop-copy 和 mark - sweep。前者需要两个堆，当需要回收的时候，中断程序，将程序复制到另一个堆，这样就避免了内存间断。而标记清扫则多用于知道不会产生太多垃圾的情况，它无法消除内存的不连续性。

——————————

**更新日志**

2018-07-27 添加了域初始化的内容